In [1]:
import pandas as pd
import numpy as np

## Intersección de usuarios (**con imágenes**) entre ciudades para domain adaptation

In [4]:
cities = ["Gijon", "Oviedo", "Barcelona", "Warsaw", "Budapest", "Hamburg", "Vienna", "Bucharest", "New York", "Paris", "Rome", "Madrid", "Berlin", "London", "Istanbul", "Moscow", "St.Petersburg", "Athens"]

In [5]:

cities_data = {}

print(f"{'':10s}\tcategory\titems\treviews\tusers")

for city in cities:
    city = city.lower().replace(" ", "")
    city_path = f"out/{city}"

    cities_data[city] = {"restaurants":{}, "pois":{}}

    for category in ["restaurants", "pois"]:
        category_path = f"{city_path}/{category}"

        item_path = f"{category_path}/items.pkl"
        review_path = f"{category_path}/reviews.pkl"
        user_path = f"{category_path}/users.pkl"

        items = pd.read_pickle(item_path)
        reviews = pd.read_pickle(review_path)
        users = pd.read_pickle(user_path)

        reviews["n_images"] = reviews["images"].apply(lambda x: len(x))
        users_with_images = reviews.loc[reviews.n_images>0]["userId"].unique()

        cities_data[city][category] = {"items":items, "reviews":reviews, "users":users, "users_images":users_with_images}
        
        print(f"{city:10s}\t{category:15s}\t{len(items)}\t{len(reviews)}\t{len(users)}")

          	category	items	reviews	users
gijon     	restaurants    	840	80753	41980
gijon     	pois           	307	17590	9916
barcelona 	restaurants    	9523	1438248	771555
barcelona 	pois           	4790	660581	314395
warsaw    	restaurants    	3606	197274	108006
warsaw    	pois           	1384	122357	60872
budapest  	restaurants    	3869	565005	330198
budapest  	pois           	2407	616166	218895
hamburg   	restaurants    	3287	190865	102362
hamburg   	pois           	1177	129185	71664
vienna    	restaurants    	4801	490952	250106
vienna    	pois           	2215	348542	131841
bucharest 	restaurants    	2614	143560	82002
bucharest 	pois           	1315	78805	45238
newyork   	restaurants    	10129	1256917	572848
newyork   	pois           	5637	1414372	534874


In [28]:
city_names = "\t".join(cities_data.keys())
print(f'city\t{city_names}')
      
for c_urs in cities_data.keys():
  print(c_urs, end="\t")
  for o_urs in cities_data.keys():
    c_data = set(cities_data[c_urs]["restaurants"]["users_images"])
    o_data = set(cities_data[o_urs]["restaurants"]["users_images"])
    print(len(c_data.intersection(o_data)), end="\t")
  print()

city	gijon	barcelona	warsaw	budapest	hamburg	vienna	bucharest	newyork	paris	rome	madrid	berlin	london
gijon	10070	661	51	157	26	131	32	182	252	407	1911	165	266	
barcelona	661	178220	1293	3868	1101	3489	863	5251	9565	10658	11361	3955	8907	
warsaw	51	1293	25988	1422	308	1093	521	736	1238	1661	1082	1243	1507	
budapest	157	3868	1422	81719	674	5253	1102	2318	3978	5365	3038	3028	4834	
hamburg	26	1101	308	674	20043	1169	161	778	1153	1019	673	2402	1356	
vienna	131	3489	1093	5253	1169	55469	800	2329	4126	4874	2780	3349	4309	
bucharest	32	863	521	1102	161	800	20430	498	915	1411	754	651	1091	
newyork	182	5251	736	2318	778	2329	498	98832	7510	6527	4114	2797	9036	
paris	252	9565	1238	3978	1153	4126	915	7510	184542	11711	6298	4355	12981	
rome	407	10658	1661	5365	1019	4874	1411	6527	11711	281425	7835	4736	10619	
madrid	1911	11361	1082	3038	673	2780	754	4114	6298	7835	139702	3099	6073	
berlin	165	3955	1243	3028	2402	3349	651	2797	4355	4736	3099	62240	5122	
london	266	8907	1507	4834	1356	4309	1091	9036

In [30]:
city_names = "\t".join(cities_data.keys())
print(f'city\t{city_names}')
      
for c_urs in cities_data.keys():
  print(c_urs, end="\t")
  for o_urs in cities_data.keys():
    c_data = set(cities_data[c_urs]["restaurants"]["users"]["userId"].unique())
    o_data = set(cities_data[o_urs]["pois"]["users"]["userId"].unique())
    print(len(c_data.intersection(o_data)), end="\t")
  print()

city	gijon	barcelona	warsaw	budapest	hamburg	vienna	bucharest	newyork	paris	rome	madrid	berlin	london
gijon	4852	859	161	575	131	321	94	699	723	685	3171	815	1165	
barcelona	907	100012	3724	14357	3791	10360	2320	22671	27229	22444	26570	18557	33254	
warsaw	83	2559	21909	4550	1023	3014	1363	3017	3252	3005	2520	4560	4944	
budapest	235	8876	4383	89333	2535	14509	3144	10809	12071	11140	8105	13759	18173	
hamburg	55	2394	952	2409	24454	3482	478	3474	3251	2561	1875	8347	4956	
vienna	203	8139	3298	17387	4543	62571	2217	10999	13357	11372	7906	14677	16684	
bucharest	61	1826	1474	3514	612	2246	16397	1976	2370	2487	1788	2608	3448	
newyork	274	16237	3125	12129	3197	10127	1820	194619	32629	23485	14404	16592	44235	
paris	414	22249	4033	16185	4244	14135	2616	34720	164519	29939	18683	21414	50372	
rome	573	24128	4765	20129	4053	15526	3626	31724	38717	150023	21524	23171	44215	
madrid	2505	21468	3286	11242	2642	8459	1974	17485	20007	16364	106770	15393	23721	
berlin	261	9470	3902	11753	8668	10779	1907	13493	

In [ ]:
def fotos_por_restaurante(reviews, city):
  data = reviews.groupby("restaurantId").n_img.sum().reset_index()
  histo = sbs.histplot(data, x="n_img", kde=False, bins=range(1,22,1));
  histo.set_xticks(np.arange(1,21,1)+.5); # Etiquetas del eje X
  labels = list(map(str,range(1,20)))+["≥20"]
  histo.set_xticklabels(labels)
  histo.set_xlabel("Número de fotos")
  histo.set_ylabel("Número de restaurantes")
  plt.tight_layout()
  histo.figure.savefig(f"{city}_hist_fotos_pr_rst.pdf")
  plt.clf()
  
def fotos_por_review(reviews, city):
  return reviews
def fotos_por_usuario(reviews, city):
  return reviews
def reviews_por_restaurante(reviews, city):
  data = reviews.groupby("restaurantId").reviewId.count().reset_index()
  data.loc[data.reviewId>=20, "reviewId"]=20

  histo = sbs.histplot(data, x="reviewId", kde=False, bins=range(1,22,1));
  histo.set_xticks(np.arange(1,21,1)+.5); # Etiquetas del eje X
  labels = list(map(str,range(1,20)))+["≥20"]
  histo.set_xticklabels(labels)
  histo.set_xlabel("Número de reseñas")
  histo.set_ylabel("Número de restaurantes")
  plt.tight_layout()
  histo.figure.savefig(f"{city}_hist_rvws_pr_rst.pdf")

def reviews_por_usuario(reviews, city):
  return reviews

In [ ]:
cities = ["gijon", "barcelona", "paris", "madrid", "newyorkcity", "london"]
# cities = ["gijon"]

base_path = "/content/drive/Shareddrives/Media"

for city in cities:
  data_path = f"/content/drive/Shareddrives/DataSets/TripAdvisor/{city}/"
  reviews = pd.read_pickle(data_path+"reviews.pkl")

  # reviews["n_img"] = reviews.images.apply(lambda x: len(x))
  restaurants = pd.read_pickle(data_path+"restaurants.pkl")
  users = pd.read_pickle(data_path+"users.pkl")
  
  # dates = pd.to_datetime(reviews.date, format="%d de %B de %Y" )
  # print(dates.min(), dates.max())

  # n_images  = reviews.n_img.sum()
  # dt = [city, len(reviews), len(users), len(restaurants), n_images]
  # print("\\"+" & ".join(map(str, dt))+" \\\\")

  reviews = reviews[["date", "images", "language", "rating", "restaurantId", "reviewId", "text", "title", "url", "userId"]]
  users = users[["id", "name", "location"]]
  restaurants = restaurants[["id", "name", "city", "priceInterval", "url", "rating"]]

  reviews.iloc[:50].to_csv(f"{base_path}/{city}_revs.csv", encoding="utf-8")
  users.iloc[:50].to_csv(f"{base_path}/{city}_usrs.csv", encoding="utf-8")
  restaurants.iloc[:50].to_csv(f"{base_path}/{city}_rsts.csv", encoding="utf-8")

In [ ]:
reviews["text"] = reviews.text.str.lower().str.split(" ")

In [ ]:
reviews.text.str.lower().str.contains("definitivas").value_counts()

AttributeError: ignored

In [ ]:
cities = {"gijon":0.3}#, "barcelona":1.6, "madrid":2.1, "newyorkcity":8.3, "paris":8.9, "london":3.2} # Ciudades con su población
stats = []

for city in cities.keys():
  data_path = f"/content/drive/Shareddrives/DataSets/TripAdvisor/{city}/"

  reviews = pd.read_pickle(data_path+"reviews.pkl")
  # restaurants = pd.read_pickle(data_path+"restaurants.pkl")
  # users = pd.read_pickle(data_path+"users.pkl")

  reviews["n_img"] = reviews.images.apply(lambda x: len(x))

  n_reviews = len(reviews.reviewId.unique())
  n_images  = reviews.n_img.sum()
  n_users = len(reviews.userId.unique())
  n_rest = len(reviews.restaurantId.unique())

  stats.append((f"\{city}", cities[city], n_reviews, n_images, n_users, n_rest))

  fotos_por_restaurante(reviews, data_path+city)
  reviews_por_restaurante(reviews, data_path+city)

stats = pd.DataFrame(stats, columns=["Conjunto", "Población", "Reseñas", "Imágenes", "Usuarios", "Restaurantes"])
print(stats.to_latex(index=False, escape=False))

In [ ]:
data = reviews.groupby("restaurantId").n_img.sum().reset_index()
data.loc[data.n_img>=20, "n_img"]=20

histo = sbs.histplot(data, x="n_img", kde=False, bins=range(1,22,1));
histo.set_xticks(np.arange(1,21,1)+.5); # Etiquetas del eje X
labels = list(map(str,range(1,20)))+["≥20"]
histo.set_xticklabels(labels)
histo.set_xlabel("Número de fotos")
histo.set_ylabel("Número de restaurantes")
plt.tight_layout()


In [ ]:
# Datos para Bea (con menos columnas)
data_sm = reviews[["reviewId", "userId", "restaurantId", "date", "images", "rating", "url"]]
data_sm.to_pickle(data_path+"reviews_sm.pkl")
